# Communicate a data set

**project for Udacity Nanodegree by Léon de Jong**

Investigating the Prosper Load dataset.

In [1]:
#import necessary modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib import rcParams
%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [2]:
df_loan = pd.read_csv('prosperLoanData.csv')

In [ ]:
#necessary transformations
#selection of variables for further analysis
voi = ['ListingKey','ListingCreationDate','Term','LoanStatus','BorrowerAPR','BorrowerRate','LenderYield','EstimatedEffectiveYield','EstimatedLoss','EstimatedReturn','ProsperRating (numeric)','ProsperRating (Alpha)','ProsperScore','ListingCategory (numeric)','CurrentlyInGroup','GroupKey','CreditScoreRangeLower','CreditScoreRangeUpper','AmountDelinquent','DebtToIncomeRatio','IncomeVerifiable','StatedMonthlyIncome','LoanKey','TotalProsperLoans','MonthlyLoanPayment']
# copied dataset with selection of variables for further wrangling and 
df_loan_filtered = df_loan[voi]
#convert listing creation date to datetime
df_loan_filtered['ListingCreationDate'] = df_loan_filtered['ListingCreationDate'].astype('datetime64')
df_loan_filtered = df_loan_filtered[df_loan_filtered['ListingCreationDate']>'2009-08-01']
# convert ProsperRating into ordered categorical types
ordinal_var_dict = {'ProsperRating (Alpha)': ['AA','A','B','C','D','E','HR']}

for var in ordinal_var_dict:
    ordered_var = pd.api.types.CategoricalDtype(ordered = True,
                                                categories = ordinal_var_dict[var])
    df_loan_filtered[var] = df_loan_filtered[var].astype(ordered_var)

#create rating list to allow for reversal of order to compare to prosperscore
Rating_list = ['AA','A','B','C','D','E','HR']

#creating a new category for AmountDelinquent
#conditions for new categories
conditions = [
    (df_loan_filtered['AmountDelinquent'] == 0),
    (df_loan_filtered['AmountDelinquent'] >0) & (df_loan_filtered['AmountDelinquent'] <=100),
    (df_loan_filtered['AmountDelinquent'] > 100),
    (df_loan_filtered['AmountDelinquent'].isnull())
]
#names for categories
values = ['None', 'Small','Large','Unknown' ]
#populate new column
df_loan_filtered['HasDelinquency'] = np.select(conditions,values)

df_loan_filtered['HasDelinquency'] = df_loan_filtered.HasDelinquency.astype('category')

#creating a new category for LoanStatus
#conditions for new categories
conditions = [
    (df_loan_filtered['LoanStatus'] == 'Current')|(df_loan_filtered['LoanStatus'] =='Completed') | (df_loan_filtered['LoanStatus'] == 'FinalPaymentInProgress'),
    (df_loan_filtered['LoanStatus'] == 'Defaulted') | (df_loan_filtered['LoanStatus'] == 'Chargedoff')|(df_loan_filtered['LoanStatus'].str.contains("Past Due"))
]
#names for categories
values = ['Nominal','Problematic' ]
#populate new column
df_loan_filtered['Loan_Result'] = np.select(conditions,values)
#restructure as categorical data
df_loan_filtered['Loan_Result'] = df_loan_filtered.Loan_Result.astype('category')

## Inital investigation of Dataset

## Show correlation between Scores, ratings and expected indicators

In [8]:
#Reverse rating order in order to keep it in line with prosperscore order
order = reversed(Rating_list)
fig, axs = plt.subplots(2,4, figsize = (22,7))
sb.barplot(ax = axs[0,0], x = df_loan_filtered['ProsperScore'],y = df_loan_filtered['AmountDelinquent'])
sb.barplot(ax = axs[1,0], x = df_loan_filtered['ProsperRating (Alpha)'], y = df_loan_filtered['AmountDelinquent'], order = order)
sb.barplot(ax = axs[0,1], x = df_loan_filtered['ProsperScore'],y = df_loan_filtered['CreditScoreRangeLower'])
sb.barplot(ax = axs[1,1], x = df_loan_filtered['ProsperRating (Alpha)'], y = df_loan_filtered['CreditScoreRangeLower'], order = reversed(Rating_list))
sb.barplot(ax = axs[0,2], x = df_loan_filtered['ProsperScore'],y = df_loan_filtered['DebtToIncomeRatio'])
sb.barplot(ax = axs[1,2], x = df_loan_filtered['ProsperRating (Alpha)'], y = df_loan_filtered['DebtToIncomeRatio'], order = reversed(Rating_list))
sb.barplot(ax = axs[0,3], x = df_loan_filtered['ProsperScore'],y = df_loan_filtered['EstimatedLoss'])
sb.barplot(ax = axs[1,3], x = df_loan_filtered['ProsperRating (Alpha)'], y = df_loan_filtered['EstimatedLoss'], order = reversed(Rating_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

## Show skewdness in amounts delinquent. 
categorical indicator would be more usefulle

In [ ]:
plt.hist(x = df_loan_filtered['AmountDelinquent'], bins=[0,1,5,10,100,1000,10000,100000,1000000])
plt.xscale('log')

Categorical indicator shows little difference in yields

In [ ]:
sb.catplot(data = df_loan_filtered, x = 'ProsperRating (Alpha)', y = 'LenderYield', hue = 'HasDelinquency',  kind = 'bar', hue_order = ['Large','Small','None']),
sb.catplot(data = df_loan_filtered, x = 'ProsperScore', y = 'LenderYield', hue = 'HasDelinquency',  kind = 'bar', hue_order = ['Large','Small','None'])

Estimated loss has a strong correlation it seems

In [ ]:
fig, axs= plt.subplots(2)
sb.regplot(ax=axs[0],y = df_loan_filtered['EstimatedLoss'], x = df_loan_filtered['ProsperScore'], x_jitter=0.2, fit_reg=True)
#using numeric values to allow for fit_reg
sb.regplot(ax=axs[1],y = df_loan_filtered['EstimatedLoss'], x = df_loan_filtered['ProsperRating (numeric)'], x_jitter=0.2, fit_reg=True)